In [1]:
import os 
import numpy as np 
import pandas as pd 
import seaborn as sns 
import matplotlib.pyplot as plt

In [2]:
plays = pd.read_csv('plays.csv')
players = pd.read_csv('players.csv')
playerplay = pd.read_csv('player_play.csv')
df = pd.read_csv('tracking_week_1.csv')

In [3]:
playerposition = players[['nflId','position']]

In [4]:
filteredtrack = df[['gameId','playId','frameId','nflId','displayName','x','y','event']]

In [5]:
filteredtrack = pd.merge(
    filteredtrack,
    playerposition,
    how='left',
    left_on=['nflId'],
    right_on=['nflId']
)

In [6]:
filteredtrack = filteredtrack[(filteredtrack['position']=="WR") | (filteredtrack['position']=="TE") | (filteredtrack['position']=="RB")]

In [10]:
filteredtrack

,gameId,playId,frameId,nflId,displayName,x,y,event,position
815,2022091200,64,1,42412.0,Tyler Lockett,32.49,22.32,huddle_break_offense,WR
816,2022091200,64,2,42412.0,Tyler Lockett,32.58,22.06,NaN,WR
817,2022091200,64,3,42412.0,Tyler Lockett,32.69,21.79,NaN,WR
818,2022091200,64,4,42412.0,Tyler Lockett,32.81,21.50,NaN,WR
819,2022091200,64,5,42412.0,Tyler Lockett,32.95,21.21,NaN,WR
...,...,...,...,...,...,...,...,...,...
7103995,2022090800,3696,171,53678.0,Ben Skowronek,3.01,50.29,pass_outcome_incomplete,WR
7103996,2022090800,3696,172,53678.0,Ben Skowronek,2.71,50.21,NaN,WR
7103997,2022090800,3696,173,53678.0,Ben Skowronek,2.43,50.12,NaN,WR
7103998,2022090800,3696,174,53678.0,Ben Skowronek,2.17,50.02,NaN,WR


In [11]:
motionplayers = playerplay[playerplay['motionSinceLineset']==True]

In [12]:
motionplayers[['gameId','playId','nflId','motionSinceLineset','inMotionAtBallSnap']]

,gameId,playId,nflId,motionSinceLineset,inMotionAtBallSnap
2,2022090800,56,42489,True,False
28,2022090800,80,47857,True,True
53,2022090800,101,53079,True,True
119,2022090800,191,53079,True,False
138,2022090800,212,47879,True,True
...,...,...,...,...,...
353695,2022110700,2261,43362,True,False
353740,2022110700,2307,43362,True,False
353863,2022110700,2496,44879,True,False
354253,2022110700,3059,54604,True,False


In [13]:
positions = players[['nflId','position']]

In [14]:
motionplayers = pd.merge(
    motionplayers,
    positions,
    how = "left",
    left_on=['nflId'],
    right_on=['nflId']
)

In [15]:
motionplayers = motionplayers[(motionplayers['position']=="WR") | (motionplayers['position']=="TE") | (motionplayers['position']=="RB")]
motionplayers

In [17]:
motiontracking = pd.merge(
    filteredtrack,
    motionplayers[['gameId','playId','nflId','motionSinceLineset','inMotionAtBallSnap']],
    how="inner",
    left_on=['gameId','playId','nflId'],
    right_on=['gameId','playId','nflId']
)


In [18]:
motiontracking

,gameId,playId,frameId,nflId,displayName,x,y,event,position,motionSinceLineset,inMotionAtBallSnap
0,2022091200,64,1,46189.0,Will Dissly,33.97,25.45,huddle_break_offense,TE,True,False
1,2022091200,64,2,46189.0,Will Dissly,34.01,25.45,NaN,TE,True,False
2,2022091200,64,3,46189.0,Will Dissly,34.04,25.45,NaN,TE,True,False
3,2022091200,64,4,46189.0,Will Dissly,34.08,25.45,NaN,TE,True,False
4,2022091200,64,5,46189.0,Will Dissly,34.11,25.44,NaN,TE,True,False
...,...,...,...,...,...,...,...,...,...,...,...
238782,2022090800,3674,167,43399.0,Tyler Higbee,22.14,49.91,pass_outcome_incomplete,TE,True,True
238783,2022090800,3674,168,43399.0,Tyler Higbee,22.11,49.99,NaN,TE,True,True
238784,2022090800,3674,169,43399.0,Tyler Higbee,22.07,50.04,NaN,TE,True,True
238785,2022090800,3674,170,43399.0,Tyler Higbee,22.02,50.04,NaN,TE,True,True


In [90]:
def filter_motion_to_snap(group):
    group = group.sort_values(by='frameId').reset_index(drop=True)
    line_set_indices = group[group['event'] == 'line_set'].index
    snap_indices = group[group['event'] == 'ball_snap'].index
    
    filtered_rows = []
    for start_idx in line_set_indices:
        snap_idx = snap_indices[snap_indices > start_idx].min()
        if pd.notna(snap_idx):
            filtered_rows.extend(group.loc[start_idx:snap_idx].index)
    return group.loc[filtered_rows]
filtered_df = motiontracking.groupby(['gameId', 'playId']).apply(filter_motion_to_snap).reset_index(drop=True)

C:\Users\rschw\AppData\Local\Temp\ipykernel_17156\1614609999.py:12: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  filtered_df = motiontracking.groupby(['gameId', 'playId']).apply(filter_motion_to_snap).reset_index(drop=True)


In [ ]:
# filtered_df is used for tracking/accel

In [96]:
def calculate_accel(df):
    
    accel_list = []
    temp_accel_list = []  

    for i in range(len(df) - 1):
        if df.iloc[i]["event"] == "line_set" and temp_accel_list:
            accel_list.extend(temp_accel_list)
            temp_accel_list = []  
        
        x_diff = df.iloc[i + 1]["x"] - df.iloc[i]["x"]
        y_diff = df.iloc[i + 1]["y"] - df.iloc[i]["y"]
        
        accel = (x_diff**2 + y_diff**2) / 0.1
        temp_accel_list.append(accel)

    accel_list.extend(temp_accel_list)
    accel_list.append(0) # Had to add a zero to the end to make sure column lengths the same
 
    df['Acceleration'] = accel_list
    print(accel_list)

In [97]:
calculate_accel(filtered_df)

[2.0289999999999973, 2.1200000000000063, 2.0259999999999914, 2.1200000000000117, 2.0259999999999914, 2.1200000000000117, 2.028999999999992, 2.117000000000009, 2.1169999999999733, 2.0250000000000257, 2.117000000000009, 2.1160000000000077, 2.1160000000000077, 2.0249999999999613, 2.117000000000009, 2.1200000000000063, 2.02899999999996, 2.034000000000026, 2.034000000000026, 1.9449999999999805, 1.9519999999999849, 1.7800000000000078, 1.6900000000000308, 1.6249999999999858, 1.5370000000000095, 1.3939999999999781, 1.3319999999999987, 1.2610000000000126, 1.1249999999999913, 1.0599999999999874, 1.0100000000000242, 0.820000000000009, 0.8089999999999636, 0.641000000000005, 0.6339999999999921, 0.48800000000003035, 0.4420000000000017, 0.4009999999999839, 0.2890000000000058, 0.19700000000000262, 0.1689999999999882, 0.10100000000000102, 0.08100000000000614, 0.03699999999999522, 0.026000000000005283, 0.015999999999999317, 0.004000000000001251, 0.008999999999996419, 0.001000000000001023, 0.000999999999

In [103]:
filtered_df.head(50)

,gameId,playId,frameId,nflId,displayName,x,y,event,position,motionSinceLineset,inMotionAtBallSnap,Acceleration
0,2022090800,56,76,42489.0,Stefon Diggs,87.02,27.91,line_set,WR,True,False,2.029
1,2022090800,56,77,42489.0,Stefon Diggs,87.04,28.36,NaN,WR,True,False,2.120
2,2022090800,56,78,42489.0,Stefon Diggs,87.06,28.82,NaN,WR,True,False,2.026
3,2022090800,56,79,42489.0,Stefon Diggs,87.07,29.27,NaN,WR,True,False,2.120
4,2022090800,56,80,42489.0,Stefon Diggs,87.09,29.73,NaN,WR,True,False,2.026
5,2022090800,56,81,42489.0,Stefon Diggs,87.10,30.18,NaN,WR,True,False,2.120
6,2022090800,56,82,42489.0,Stefon Diggs,87.12,30.64,NaN,WR,True,False,2.029
7,2022090800,56,83,42489.0,Stefon Diggs,87.14,31.09,NaN,WR,True,False,2.117
8,2022090800,56,84,42489.0,Stefon Diggs,87.15,31.55,NaN,WR,True,False,2.117
9,2022090800,56,85,42489.0,Stefon Diggs,87.16,32.01,NaN,WR,True,False,2.025
